In [1]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
# tokenize the dataset
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize('What is the capital of Germany?')

['what', 'is', 'the', 'capital', 'of', 'germany']

In [4]:
# vocabulary
vocab = {'<UNK>':0}

In [5]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

  print(merged_tokens)

In [6]:
df.apply(build_vocab, axis = 1)

['what', 'is', 'the', 'capital', 'of', 'france', 'paris']
['what', 'is', 'the', 'capital', 'of', 'germany', 'berlin']
['who', 'wrote', 'to', 'kill', 'a', 'mockingbird', 'harper-lee']
['what', 'is', 'the', 'largest', 'planet', 'in', 'our', 'solar', 'system', 'jupiter']
['what', 'is', 'the', 'boiling', 'point', 'of', 'water', 'in', 'celsius', '100']
['who', 'painted', 'the', 'mona', 'lisa', 'leonardo-da-vinci']
['what', 'is', 'the', 'square', 'root', 'of', '64', '8']
['what', 'is', 'the', 'chemical', 'symbol', 'for', 'gold', 'au']
['which', 'year', 'did', 'world', 'war', 'ii', 'end', '1945']
['what', 'is', 'the', 'longest', 'river', 'in', 'the', 'world', 'nile']
['what', 'is', 'the', 'capital', 'of', 'japan', 'tokyo']
['who', 'developed', 'the', 'theory', 'of', 'relativity', 'albert-einstein']
['what', 'is', 'the', 'freezing', 'point', 'of', 'water', 'in', 'fahrenheit', '32']
['which', 'planet', 'is', 'known', 'as', 'the', 'red', 'planet', 'mars']
['who', 'is', 'the', 'author', 'of', '19

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [7]:
len(vocab)

324

In [8]:
# convert words to numerical indices
def text_to_indices(text, vocab):
  indexed_text = []

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [9]:
text_to_indices('What is transformer', vocab)

[1, 2, 0]

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [12]:
dataset = QADataset(df, vocab)

In [13]:
dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [14]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [15]:
for question , answer in dataloader:
  print(question, answer)

tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor([[199]])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([[188]])
tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([[99]])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([[260]])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([[124]])
tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([[307]])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([[52]])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([[121]])
tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([[295]])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([[316]])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([[205]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([[233]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor

In [16]:
import torch.nn as nn

In [17]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [18]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [19]:
x = nn.Embedding(330, embedding_dim=50)

In [20]:
a = x(dataset[0][0])

In [21]:
a

tensor([[-1.1321,  0.0577, -1.3952, -1.4694, -0.3391, -1.0252,  1.2401, -0.4959,
         -2.8411, -0.8803,  0.5460, -1.3759, -0.4237, -0.7808, -0.2029,  0.0983,
         -0.2088,  0.4787,  0.1558, -0.5461, -0.7556,  0.0787,  0.0142, -1.5852,
          1.3602, -0.5277, -0.3850,  0.3840, -1.4967, -0.5771, -1.0179,  1.2716,
         -0.3192,  0.0622, -0.0763, -0.7493, -0.9779,  0.6055,  0.6455,  0.1201,
          0.6821,  0.3985, -0.3047, -1.5467, -0.6077,  0.8067,  0.2489,  1.1432,
          0.4449, -0.6554],
        [-0.3718, -1.2859,  1.2903,  1.0654,  0.0896,  0.7027, -0.4526,  0.3327,
          0.2497, -0.2854,  0.6703,  0.3828,  0.4687, -0.1043,  0.2145,  0.2046,
          1.1169,  1.0346,  1.7669,  0.5151,  1.5036,  0.4370, -0.5905, -1.1102,
          1.1207,  0.7832,  2.4049,  0.7481,  0.2178, -0.6416,  0.4640, -0.7941,
          0.1818, -0.9757,  0.6126, -0.4273, -0.2047, -2.8126,  0.3320,  0.6471,
          1.0469, -0.1324, -0.1218,  1.8553, -1.0518,  0.1093, -1.0803, -0.7977,


In [22]:
y = nn.RNN(50,64)

In [23]:
y(a)

(tensor([[ 0.3641, -0.1376,  0.1153,  0.4903,  0.3849,  0.0989,  0.1562,  0.5266,
          -0.7186,  0.2070,  0.4730, -0.1899,  0.1854,  0.0235, -0.1557, -0.4728,
           0.8315,  0.0749, -0.2166, -0.1797, -0.6919, -0.0848,  0.0412, -0.0344,
          -0.4857, -0.2606, -0.4574,  0.1818,  0.4927,  0.2309,  0.0514, -0.4532,
          -0.3192,  0.2922,  0.4085,  0.0636, -0.5820,  0.1914, -0.4224,  0.0191,
          -0.1748, -0.1340, -0.6995, -0.4353, -0.6629,  0.1000,  0.4199,  0.0825,
          -0.5049,  0.4666,  0.8458, -0.4064,  0.5511,  0.1629,  0.4182, -0.2245,
           0.4360, -0.4322, -0.2136,  0.5043,  0.7262, -0.3869, -0.5804,  0.2838],
         [ 0.3124, -0.3005, -0.5634, -0.5532, -0.7346,  0.4649, -0.8348, -0.8940,
           0.6979, -0.4850, -0.6024, -0.0120,  0.2220, -0.2009, -0.0115, -0.6741,
          -0.2948, -0.2965,  0.6888, -0.4391,  0.4967, -0.8518,  0.2915,  0.6102,
          -0.6735,  0.2762,  0.1277,  0.3226, -0.7815, -0.7091,  0.0621,  0.4862,
           0.23

In [24]:
y(a)[0].shape

torch.Size([6, 64])

In [25]:
b = y(a)[1]

In [26]:
z = nn.Linear(64, 330)

In [27]:
z(b).shape

torch.Size([1, 330])

In [28]:
learning_rate = 0.001
epochs = 20

In [29]:
model = SimpleRNN(len(vocab))

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [32]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [33]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # output = output.view(-1, output.size(-1))
    # answer = answer.view(-1)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 523.346014
Epoch: 2, Loss: 454.895668
Epoch: 3, Loss: 374.717069
Epoch: 4, Loss: 313.074487
Epoch: 5, Loss: 261.616409
Epoch: 6, Loss: 213.420938
Epoch: 7, Loss: 169.804452
Epoch: 8, Loss: 132.094417
Epoch: 9, Loss: 101.766497
Epoch: 10, Loss: 78.171867
Epoch: 11, Loss: 59.931057
Epoch: 12, Loss: 47.566643
Epoch: 13, Loss: 38.026240
Epoch: 14, Loss: 31.111705
Epoch: 15, Loss: 25.501053
Epoch: 16, Loss: 21.503536
Epoch: 17, Loss: 18.172937
Epoch: 18, Loss: 15.369568
Epoch: 19, Loss: 13.354370
Epoch: 20, Loss: 11.721244


In [42]:
def predict(model, question, threshold = 0.5):
  # convert text to indices
  numerical_question = text_to_indices(question, vocab)

  # convert to tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max probs
  value, index = torch.max(probs, dim=1)

  if value<threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [45]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [44]:
list(vocab.keys())[7]

'paris'